### Imports

In [1]:
import fast_stats
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix
)
import numpy as np

### Settings

In [2]:
SIZE = (10, 512, 512)
NUM_CATS = 8

### Binary compared to scikit-learn

In [3]:
y_true = np.random.randint(0, 2, SIZE).astype(bool).flatten()
y_pred = np.random.randint(0, 2, SIZE).astype(bool).flatten()

In [4]:
%%timeit
_ = precision_score(y_true, y_pred)

580 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%%timeit
_ = recall_score(y_true, y_pred)

589 ms ± 5.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
%%timeit
_ = f1_score(y_true, y_pred)

585 ms ± 6.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
# don't need to flatten them for fast-stats
y_true = np.random.randint(0, 2, SIZE).astype(bool)
y_pred = np.random.randint(0, 2, SIZE).astype(bool)

In [8]:
%%timeit
_ = fast_stats.binary_precision(y_true, y_pred)

3.51 ms ± 79 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
%%timeit
_ = fast_stats.binary_recall(y_true, y_pred)

3.19 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
%%timeit
_ = fast_stats.binary_f1_score(y_true, y_pred)

4.47 ms ± 65.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [11]:
assert np.allclose(
    fast_stats.binary_precision(y_true, y_pred),
    precision_score(y_true.flatten(), y_pred.flatten())
)
assert np.allclose(
    fast_stats.binary_recall(y_true, y_pred),
    recall_score(y_true.flatten(), y_pred.flatten())
)
assert np.allclose(
    fast_stats.binary_f1_score(y_true, y_pred),
    f1_score(y_true.flatten(), y_pred.flatten())
)

### Binary compared to numpy

In [12]:
y_true, y_pred = y_true.astype(bool), y_pred.astype(bool)

In [13]:
%%timeit
_ = np.logical_and(y_true, y_pred).sum() / y_pred.sum()

5.67 ms ± 25.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
_ = fast_stats.binary_precision(y_true, y_pred)

3.11 ms ± 35.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Multiclass compared to sklearn

In [15]:
y_true = np.random.randint(0, NUM_CATS, SIZE)
y_pred = np.random.randint(0, NUM_CATS, SIZE)

In [16]:
%%timeit
_ = precision_score(
    y_true.reshape(-1), 
    y_pred.reshape(-1), 
    labels=list(range(NUM_CATS)), 
    average=None
)

671 ms ± 3.28 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
tmp = fast_stats.precision(
    y_true,
    y_pred,
    list(range(NUM_CATS)),
)

67.7 ms ± 476 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
assert np.allclose(
    precision_score(
        y_true.reshape(-1), 
        y_pred.reshape(-1), 
        labels=list(range(NUM_CATS)), 
        average=None
    ),
    fast_stats.precision(
        y_true,
        y_pred,
        list(range(NUM_CATS)),
    )
)

In [19]:
%%timeit
_ = recall_score(
    y_true.reshape(-1), 
    y_pred.reshape(-1), 
    labels=list(range(NUM_CATS)), 
    average=None
)

674 ms ± 3.44 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%%timeit
tmp = fast_stats.recall(
    y_true,
    y_pred,
    np.array(list(range(NUM_CATS)), dtype=y_true.dtype)
)

68.3 ms ± 1.41 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
assert np.allclose(
    recall_score(
        y_true.reshape(-1), 
        y_pred.reshape(-1), 
        labels=list(range(NUM_CATS)), 
        average=None
    ),
    fast_stats.recall(
        y_true,
        y_pred,
        list(range(NUM_CATS)),
    )
)

In [22]:
%%timeit
_ = f1_score(
    y_true.reshape(-1), 
    y_pred.reshape(-1), 
    labels=list(range(NUM_CATS)), 
    average=None
)

679 ms ± 3.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [23]:
%%timeit
tmp = fast_stats.f1_score(
    y_true,
    y_pred,
    np.array(list(range(NUM_CATS)), dtype=y_true.dtype)
)

66.5 ms ± 339 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [24]:
assert np.allclose(
    f1_score(
        y_true.reshape(-1), 
        y_pred.reshape(-1), 
        labels=list(range(NUM_CATS)), 
        average=None
    ),
    fast_stats.f1_score(
        y_true,
        y_pred,
        list(range(NUM_CATS)),
    )
)

### Confusion matrix compared to sklearn

In [25]:
y_true = np.random.randint(0, NUM_CATS, SIZE).flatten()
y_pred = np.random.randint(0, NUM_CATS, SIZE).flatten()

In [26]:
%%timeit
_ = confusion_matrix(y_true, y_pred)

274 ms ± 663 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
%%timeit
# while labels is optional argument providing
# labels will lead to a significant speedup
# since it will not have to be inferred
_ = confusion_matrix(y_true, y_pred, labels = list(range(NUM_CATS)))

149 ms ± 655 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [28]:
# don't need to flatten them for fast-stats, 
# this is another point for speedup depending on use-case
y_true = np.random.randint(0, NUM_CATS, SIZE)
y_pred = np.random.randint(0, NUM_CATS, SIZE)

In [29]:
%%timeit
_ = fast_stats.confusion_matrix(y_true, y_pred)

135 ms ± 824 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [30]:
%%timeit
# while labels is optional argument providing
# labels will lead to a significant speedup
# since it will not have to be inferred
_ = fast_stats.confusion_matrix(y_true, y_pred, labels = list(range(NUM_CATS)))

67.5 ms ± 560 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [31]:
assert np.allclose(
    confusion_matrix(y_true.flatten(), y_pred.flatten(), labels = list(range(NUM_CATS))),
    fast_stats.confusion_matrix(y_true, y_pred)
)

### Effect of flattening or reshaping for scikit-learn

In [32]:
mat = np.random.randint(0, NUM_CATS, SIZE)

In [33]:
%%timeit
_ = mat.flatten()

1.71 ms ± 31.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [34]:
%%timeit
_ = mat.reshape(-1)

325 ns ± 1.53 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
